In [1]:
from scapy.all import *
import pandas as pd
import hawkeye as he
import numpy as np

In [2]:
#filepath = '../../mimir/home_big.pcap'
#filepath = './nynas.pcap'
#filepath = './ilcafe.pcap'
#filepath = './eh2.pcap'
#filepath = './10sitedump.pcap'
#filepath = './ilcafe2.pcap'
filepath = './monitor_jog.pcap'
filepath = './shortmonitor.pcap'
#filepath = '../../achotels_public.pcap'

In [3]:
_, _,df = he.tcpdump_to_dataframe(filepath)

Reading tcp into scapy...
Converting dump into list of dicts...
Normalizing dicts
Converting into dataframe...


In [6]:
"""
Check the captured packets for a src in 10.0 or 192.168
Might be better to manually inspect stuff
"""
def get_local_hosts(df):
    hosts = df['src'].unique()
    local_hosts = []
    for host in hosts:
        if host.startswith('192.168') or host.startswith('10.0'):
            local_hosts.append(host)
    return local_hosts
#local_hosts = get_local_hosts(df)
#local_hosts

In [7]:
df

StopIteration: 

StopIteration: 

In [ ]:
scapy_packets = rdpcap(filepath)


In [ ]:
scapy_packets[0]

In [ ]:
x = []
for p in scapy_packets:
    x.append((p.dst, p.src))

In [ ]:
a = pd.DataFrame(x)

a = a[a[0] != "ff:ff:ff:ff:ff:ff"]
a = a[a[1] != "ff:ff:ff:ff:ff:ff"]
print(a.shape)
a = a[a[0] != "a8:6d:aa:eb:96:a0"]
a = a[a[1] != "a8:6d:aa:eb:96:a0"]
print(a.shape)


In [ ]:
len(x)

In [ ]:
#Yes, we capture only that kind of traffic, only twelve packets were to other


In [ ]:
destinations = []
sources = []
for p in scapy_packets:
    if p.haslayer(IP):
        try:
            if p[IP].payload.dport in [80, 443, 8080]:
                destinations.append(p[IP].dst)
                sources.append(p[IP].src)
        except:
            continue

df = pd.DataFrame(data = [destinations, sources]).T

df = df.rename(columns= {0:"destination", 1:"source"})
groups = df.groupby("source")         

In [ ]:
for name, group in groups:
    print(name)

    display(group)

In [ ]:
dnsresponses = []
askers = []
for p in scapy_packets:
    if p.haslayer(DNSQR) and p.haslayer(IP):
        dnsresponses.append(p[DNSQR].qname.decode('ascii'))
        askers.append(p[IP].src)
df = pd.DataFrame(data = [dnsresponses, askers]).T

df = df.rename(columns= {0:"query", 1:"source"})
groups = df.groupby("source")
for name, group in groups:
    print(name)
    result = group.groupby('query').count()
    result = result.rename(columns= {"source": "count"})

    display(result)

In [ ]:
def match_dns_to_ip(scapy_packets):
    dnsresponses = []
    askers = []
    for p in scapy_packets:
        if p.haslayer(DNSQR) and p.haslayer(IP):
            dnsresponses = p[DNSQR].qname
            askers.append({query:p[IP].dst})

    return pd.DataFrame.from_dict(response)
match_dns_to_ip(scapy_packets).T        

In [ ]:

for h in local_hosts:
    print(h)
    print()
    visited_sites = []
    for p in scapy_packets:
        if p.haslayer(DNS) and p.haslayer(IP):
            if p[IP].fields['src'] == h:
                visited_sites.append(p[DNS].qd.qname.decode('ascii'))
    print(visited_sites)
    print(len(visited_sites))

In [ ]:
for p in scapy_packets:
    if p.haslayer(DNS) and p.haslayer(IP):
        if p[IP].fields['src'] == '192.168.50.60':
            if p[IP].fields['dst'] in local_hosts:
                #p.show()
                print(p[DNS].qd.qname.decode('ascii'))

In [ ]:
x[-1].show()

In [ ]:
tcpconnections = df[df['proto'] == 'TCP']
tcpconnections[tcpconnections['dst'] == local_hosts[1]]

In [ ]:
socket.gethostbyaddr("172.217.21.142")

# Note
www.google.com has ip 	172.217.21.142
socket.gethostbyaddr("172.217.21.142") returns arn11s02-in-f14.1e100.net

I suppose that this is because arn11s02-in-f14.1e100.net is my local google host.

# Extract DNS queries
The DNS queries are scapy objects stored in the dataframe. They cannot be displayed by the pandas dataframe, so don't ask it to.


In [ ]:
x = df[df['src'] == local_hosts[1]]['dst'].unique()

for ip in x:
    try:
        print(socket.gethostbyaddr(ip)[0])
    except:
        continue

In [ ]:
c

### Area Network Analysis
Determine subnet

Determine actors in subnet

Determine sites visited by actors in subnet